# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

customer_analysis_df = pd.read_csv(url)

In [12]:
customer_analysis_df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [14]:
low_claim_yes = customer_analysis_df[(customer_analysis_df["Total Claim Amount"] < 1000) & (customer_analysis_df["Response"] == "Yes")]
low_claim_yes.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
8,8,FM55990,California,5989.773931,Yes,Premium,College,1/19/11,Employed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,NaN
15,15,CW49887,California,4626.801093,Yes,Basic,Master,1/16/11,Employed,F,...,0.0,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,NaN
19,19,NJ54277,California,3746.751625,Yes,Extended,College,2/26/11,Employed,F,...,1.0,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A
27,27,MQ68407,Oregon,4376.363592,Yes,Premium,Bachelor,2/28/11,Employed,F,...,0.0,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,NaN


In [ ]:
yes_customer = customer_analysis_df[customer_analysis_df["Response"] == "Yes"]

In [18]:
policy_gender = yes_customer.groupby(["Policy Type", "Gender"]).agg({"Monthly Premium Auto" : "mean", "Customer Lifetime Value": "mean", "Total Claim Amount": "mean"}).reset_index()
policy_gender

,Policy Type,Gender,Monthly Premium Auto,Customer Lifetime Value,Total Claim Amount
0,Corporate Auto,F,94.301775,7712.628736,433.738499
1,Corporate Auto,M,92.188312,7944.465414,408.582459
2,Personal Auto,F,98.998148,8339.791842,452.965929
3,Personal Auto,M,91.085821,7448.383281,457.010178
4,Special Auto,F,92.314286,7691.584111,453.280164
5,Special Auto,M,86.343750,8247.088702,429.527942


In [ ]:
#Con esta tabla vemos que "Personal Auto-Female" da el valor más alto en Customer Lifetime Value, pero vemos que el Total Claim Amount no es de los más bajos. Esto implica más ingresos pero más riesgo para la empresa.
#Viendo "Corporate Auto-Male", vemos que tiene el valor más bajo de Total Claim Amount, por lo que sería la opción más segura, aunque el Lifetime Value es un poco más bajo.
#Teniendo en cuenta "Special Auto-Male", creo que seria la opción más equilibrada, respecto a todo lo que hemos comentado anteriormente.

In [20]:
customers_by_state = customer_analysis_df.groupby("State").size().reset_index(name = 'count')
states_over_500 = customers_by_state[customers_by_state["count"] > 500]
states_over_500

,State,count
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


In [22]:
edu_gender = customer_analysis_df.groupby(["Education", "Gender"]).agg({"Customer Lifetime Value": ["max", "min", "median"]})
edu_gender

Customer Lifetime Value                          
                                                max          min       median
Education            Gender                                                  
Bachelor             F                  73225.95652  1904.000852  5640.505303
                     M                  67907.27050  1898.007675  5548.031892
College              F                  61850.18803  1898.683686  5623.611187
                     M                  61134.68307  1918.119700  6005.847375
Doctor               F                  44856.11397  2395.570000  5332.462694
                     M                  32677.34284  2267.604038  5577.669457
High School or Below F                  55277.44589  2144.921535  6039.553187
                     M                  83325.38119  1940.981221  6286.731006
Master               F                  51016.06704  2417.777032  5729.855012
                     M                  50568.25912  2272.307310  5579.099207

In [24]:
edu_gender.columns = ["max_CLV", "min_CLV", "median_CLV"]
edu_gender = edu_gender.reset_index()
edu_gender

,Education,Gender,max_CLV,min_CLV,median_CLV
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5577.669457
6,High School or Below,F,55277.44589,2144.921535,6039.553187
7,High School or Below,M,83325.38119,1940.981221,6286.731006
8,Master,F,51016.06704,2417.777032,5729.855012
9,Master,M,50568.25912,2272.307310,5579.099207


In [ ]:
#Analizando esta tabla podemos ver que los hombres con educación "High School or Below", tienen el max_CLV más alto que cualquier otro, sucediendo lo mismo con la mediana. 
#Lo anteriormente razonado pero relacionado con mujeres, lo podemos ver en aquellas que tienen por educación "Bachelor"
#Por otro lado, observamos que en educaciones como "Doctor" y "Master", no destacan tanto los niveles.
#Refiriéndonos a este dataset, sacamos la conclusión de que una educación más alta no equivale a clientes más rentables. Los hombres con educación más baja ("High School or Below"), pueden ser objetivos más rentables.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here